In [60]:
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from time import sleep
import pandas as pd
import numpy as np
from openpyxl.workbook import Workbook
import base64

In [61]:
#функция, которая создает xpath для приложений в top_free и top_grossing
def xpath_top_free_and_top_grossing(first_number=1, second_number=10):
    i = first_number
    while i <= second_number:
        xpath = '//*[@id="cy-top-rows-wrap"]/div/div/top-apps-item[{:d}]/app-list-item/div[2]/div/div/a'
        xpath_ = xpath.format(i)
        i = i + 1
        top_free_and_top_grossing_xpaths.append(xpath_)

In [62]:
top_free_and_top_grossing_xpaths = [] #пустой массив, чтобы положить в него xpath для дальнейшего парсинга ссылок
xpath_top_free_and_top_grossing()
#значения в функции по умолчанию 1, 10 - т.е. сохраняется топ 10 приложений, если хочешь поменять, то введи два числа через ","
#пример xpath_top_free_and_top_grossing(10,20) - будет сохранен топ с 10 по 20 приложение

In [65]:
#функция для поиска ссылок в top grossing
def links_top_grossing_function(link):
    options = Options()
    options.add_argument("--window-size=600,600")
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    driver.get(f'{link}')
    sleep(4)
    category = driver.find_element("xpath", '//*[@id="cy-top-head-select-col"]/div')
    category.click()
    grossing = driver.find_element("xpath", '//*[@id="cy-item-revenue"]/span')
    grossing.click()
    sleep(2)
    for i in top_free_and_top_grossing_xpaths:
        xpath = driver.find_element('xpath', i)
        link = xpath.get_attribute('href')
        top_grossing_links.append(link)
    return 'Все спарсилось. Линки хранятся в переменной top_grossing_links'

In [66]:
#функция для поиска ссылок в top free
def links_top_free_function(link):
    options = Options()
    options.add_argument("--window-size=600,600")
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    driver.get(f'{link}')
    sleep(4)
    for i in top_free_and_top_grossing_xpaths:
        xpath = driver.find_element('xpath', i)
        link = xpath.get_attribute('href')
        top_free_links.append(link)
    return 'Все спарсилось. Линки хранятся в переменной top_free_links'

In [69]:
%%time
top_free_links = [] #пустой массив, где будут хранится ссылки
links_top_free_function('https://appmagic.rocks/top-charts/apps?tag=243572') #ccылка откуда парсим, не забудь ''

CPU times: total: 0 ns
Wall time: 11.4 s


'Все спарсилось. Линки хранятся в переменной top_free_links'

In [68]:
%%time
top_grossing_links = [] #пустой массив, где будут хранится ссылки
links_top_grossing_function('https://appmagic.rocks/top-charts/apps?tag=243572') #ccылка откуда парсим, не забудь ''

CPU times: total: 15.6 ms
Wall time: 14.3 s


'Все спарсилось. Линки хранятся в переменной top_grossing_links'

In [70]:
# делаем пустые массива для сбора информации в top_free
name_free = []
publisher_free = []
setting_free = []
revenue_free = []
lifetime_revenue_free = []
downloads_free = []
lifetime_downloads_free = []
rpd_free = []
realeses_free = []
raiting_free = []

In [71]:
# делаем пустые массива для сбора информации в top_grossing
name_grossing = []
publisher_grossing = []
setting_grossing = []
revenue_grossing = []
lifetime_revenue_grossing = []
downloads_grossing = []
lifetime_downloads_grossing = []
rpd_grossing = []
realeses_grossing = []
raiting_grossing = []

In [72]:
#функция для парсинга top grossing
def parcing_gross():
    for i in top_grossing_links:
        #опция, чтобы браузер не открывался
        options = Options()
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)
        sleep(3)
        driver.get(i)
        #чтобы сайт успел прогрузится ждем 4 секунды перед парсингом
        sleep(6)
        #берем название игры
        name = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[1]/div[1]/div[2]/div[1]/div')
        name = name.text
        name_grossing.append(name)
        #берем паблишера
        publisher = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[1]/div[1]/div[2]/div[2]')
        publisher = publisher.text
        publisher_grossing.append(publisher)
        #пытаемся найти сетинг на странице
        try: 
            setting = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[1]/div[1]/div[2]/application-tags/div[4]/div[2]')
            setting = setting.text
        except NoSuchElementException:
            setting = '-'
        setting_grossing.append(setting)
        #берем ревенью за 30 дней
        revenue = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div/div')
        if revenue.text == 'No revenue over lifetime':
            revenue = 'No revenue over lifetime'
        else: 
            revenue = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div[1]/span/span')
            revenue = revenue.text  
            sleep(1)
            #находим и кликаем на суммарное ревенью
            lifetime_revenue = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div[1]/div/div[2]')
            lifetime_revenue.click()
            sleep(1)
            #далее берем информацию о lifetime revenue
            lifetime_revenue_value = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div[1]/span/span')
            lifetime_revenue_value = lifetime_revenue_value.text
        revenue_grossing.append(revenue)
        lifetime_revenue_grossing.append(lifetime_revenue_value)
        #берем установки за 30 дней
        downloads = driver.find_element("xpath", "/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget")
        if downloads.text == 'No downloads over lifetime':
            downloads = 'No downloads over lifetime'
            lifetime_downloads_value = 'No downloads over lifetime'
        else:
            downloads = driver.find_element("xpath", "/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget/div/div[1]/span/span")
            downloads = downloads.text
            sleep(1)
            #находим и кликаем на суммарные скачивания
            lifetime_downloads = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget/div/div[1]/span/span')
            lifetime_downloads.click()
            sleep(1)
            #берем информацию о суммарных скачиваниях
            lifetime_downloads_value = driver.find_element("xpath", "/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget/div/div[1]/span/span")
            lifetime_downloads_value = lifetime_downloads_value.text
        downloads_grossing.append(downloads)
        lifetime_downloads_grossing.append(lifetime_downloads_value)
        #берем rpd
        rpd = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[2]/div[2]/ltv-widget')
        if rpd.text == 'RpD — $0':
            rpd = '$0'
        else:
            rpd = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[2]/div[2]/ltv-widget/div[2]')
            rpd = rpd.text
            rpd = rpd.replace('\n', ' ')
        rpd_grossing.append(rpd)
        #берем дату релиза
        realeses = driver.find_element("xpath", '//*[@id="mat-tab-content-0-0"]/div/info/div[3]/app-description/div[1]/span[2]')
        realeses = realeses.text
        realeses_grossing.append(realeses)
        #берем общий рейтинг
        try:
            raiting = driver.find_element("xpath", '//*[@id="mat-tab-content-0-0"]/div/info/app-rate-info/div[1]/div[1]')
            raiting = raiting.text
        except NoSuchElementException:
            raiting = '-'
        raiting_grossing.append(raiting)

In [73]:
#функция для парсинга top free
def parcing_free():
    for i in top_free_links:
        #опция, чтобы браузер не открывался
        options = Options()
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)
        sleep(3)
        driver.get(i)
        #чтобы сайт успел прогрузится ждем 4 секунды перед парсингом
        sleep(6)
        #берем название игры
        name = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[1]/div[1]/div[2]/div[1]')
        name = name.text
        name_free.append(name)
        #берем паблишера
        publisher = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[1]/div[1]/div[2]/div[2]')
        publisher = publisher.text
        publisher_free.append(publisher)
        #пытаемся найти сетинг на странице
        try: 
            setting = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[1]/div[1]/div[2]/application-tags/div[4]/div[2]')
            setting = setting.text
        except NoSuchElementException:
            setting = '-'
        setting_free.append(setting)
        #берем ревенью за 30 дней
        revenue = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div/div')
        if revenue.text == 'No revenue over lifetime':
            revenue = 'No revenue over lifetime'
            lifetime_revenue_value = 'No revenue over lifetime'
        else: 
            revenue = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div[1]/span/span')
            revenue = revenue.text  
            sleep(1)
            #находим и кликаем на суммарное ревенью
            lifetime_revenue = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div[1]/div/div[2]')
            lifetime_revenue.click()
            #далее берем информацию о lifetime revenue
            lifetime_revenue_value = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div[1]/span/span')
            lifetime_revenue_value = lifetime_revenue_value.text
        revenue_free.append(revenue)
        lifetime_revenue_free.append(lifetime_revenue_value)
        #берем установки за 30 дней
        downloads = driver.find_element("xpath", "/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget")
        if downloads.text == 'No downloads over lifetime':
            downloads = 'No downloads over lifetime'
            lifetime_downloads_value = 'No downloads over lifetime'
        else:
            downloads = driver.find_element("xpath", "/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget/div/div[1]/span/span")
            downloads = downloads.text
            #находим и кликаем на суммарные скачивания
            lifetime_downloads = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget/div/div/div/div[2]')
            lifetime_downloads.click()
            #берем информацию о суммарных скачиваниях
            lifetime_downloads_value = driver.find_element("xpath", "/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget/div/div[1]/span/span")
            lifetime_downloads_value = lifetime_downloads_value.text
        downloads_free.append(downloads)
        lifetime_downloads_free.append(lifetime_downloads_value)
        #берем rpd
        rpd = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[2]/div[2]/ltv-widget')
        if rpd.text == 'RpD — $0':
            rpd = '$0'
        else:
            rpd = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[2]/div[2]/ltv-widget/div[2]')
            rpd = rpd.text
            rpd = rpd.replace('\n', ' ')
        rpd_free.append(rpd)
        #берем дату релиза
        realeses = driver.find_element("xpath", '//*[@id="mat-tab-content-0-0"]/div/info/div[3]/app-description/div[1]/span[2]')
        realeses = realeses.text
        realeses_free.append(realeses)
        #берем общий рейтинг
        try:
            raiting = driver.find_element("xpath", '//*[@id="mat-tab-content-0-0"]/div/info/app-rate-info/div[1]/div[1]')
            raiting = raiting.text
        except NoSuchElementException:
            raiting = '-'
        raiting_free.append(raiting)
        driver.close()

In [74]:
%%time
#запускаем парсинг top_free
parcing_free()

CPU times: total: 93.8 ms
Wall time: 3min


In [75]:
%%time
#запускаем парсинг top_grossing, p.s. лучше запускать по очереди
parcing_gross()

CPU times: total: 188 ms
Wall time: 3min 47s


In [76]:
#создаем датафрейм top free
df_top_free = pd.DataFrame({'Game_Name': name_free, 'Publisher':publisher_free, "Setting":setting_free, \
                  "Revenue": revenue_free, "Lifetime_revenue": lifetime_revenue_free, "Downloads":downloads_free, \
                  "Lifetime_downloads": lifetime_downloads_free, "Rpd": rpd_free, "Realese_date":realeses_free, "Raiting": raiting_free})             

In [77]:
#создаем датафрейм top grossing
df_top_grossing = pd.DataFrame({'Game_Name': name_grossing, 'Publisher':publisher_grossing, "Setting":setting_grossing, \
                  "Revenue": revenue_grossing, "Lifetime_revenue": lifetime_revenue_grossing, "Downloads":downloads_grossing, \
                  "Lifetime_downloads": lifetime_downloads_grossing, "Rpd": rpd_grossing, "Realese_date":realeses_grossing, "Raiting": raiting_grossing})             

In [83]:
#грузим в файл excel с названием 'top_free'
df_top_free.to_excel('top_free.xlsx', sheet_name='top_free', index=False)

In [84]:
#грузим в файл excel с названием 'top_grossing'
df_top_grossing.to_excel('top_grossing.xlsx', sheet_name = 'top_grossing', index=False)